In [3]:
import torchvision.datasets as dset
import torch
import transforms as T
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from utils import collate_fn

from engine import train_one_epoch, evaluate

from PIL import Image
import os
import os.path
from torchvision.datasets.vision import VisionDataset


class MyCocoDetection(VisionDataset):
    """`MS Coco Detection <http://mscoco.org/dataset/#detections-challenge2016>`_ Dataset.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(self, root, annFile, transform=None, target_transform=None, transforms=None):
        super(MyCocoDetection, self).__init__(root, transforms, transform, target_transform)
        from pycocotools.coco import COCO
        self.coco = COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: Tuple (image, target). target is the object returned by ``coco.loadAnns``.
        """
        coco = self.coco
        img_id = self.ids[index]
        ann_ids = coco.getAnnIds(imgIds=img_id)
        target = coco.loadAnns(ann_ids)

        
        #path = coco.loadImgs(img_id)[0]['file_name']
        path = '/home/test/NightPedestrianDetection/Code/Preliminary_tests/FasterRCNNEvaluationTutorial/people.jpg'

        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        '''
        print('target to return: ')
        print(target)
        print(type(target))
        print('target len : ' + str(len(target)))
        '''
        
        return img, target


    def __len__(self):
        return len(self.ids)



def get_instance_segmentation_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

def get_dataloader(path2data,path2json,batch_size,num_workers):
    transforms = []
    transforms.append(T.ToTensor())
    transform = T.Compose(transforms)
    a = T.ToTensor()
    
    coco_train = MyCocoDetection(path2data,path2json, transforms = a)

    print('Number of samples: ', len(coco_train))
    
    data_loader = torch.utils.data.DataLoader(coco_train, batch_size=batch_size, num_workers=num_workers, 
                                              drop_last=True,shuffle=True,
                                                  collate_fn=collate_fn)
    return data_loader

def main():
    device = torch.device("cuda:0")
    real_coco_val = '/home/test/tmp/an/annotations/instances_val2014.json'
    path2data='/home/test/data/nightowls/nightowls_validation/'
    path2json='/home/test/data/nightowls/nightowls_validation.json'
    batch_size = 2
    num_workers = 2
    num_epochs = 1

    data_loader = get_dataloader(path2data,path2json,batch_size,num_workers)

    model = get_instance_segmentation_model(4)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,momentum=0.9, weight_decay=0.0005)
    

    #for i, (imgs,targets) in enumerate(data_loader):
        #print(i)

        #if(i < 5):
            #print('targets:')
            #print(targets)
            
        #for t in targets:
            #print('target ' + str(type(t)))
            #print(t)
    
    #targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    
    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=1)

    
        
main()
    

loading annotations into memory...
Done (t=0.30s)
creating index...
index created!
Number of samples:  51848


AttributeError: 'list' object has no attribute 'items'